In [ ]:
!pip install easyocr
!pip install opencv-python-headless==4.1.2.30
!pip install Pillow

In [81]:
#!pip install easyocr
#!pip install opencv-python-headless==4.1.2.30
#!pip install Pillow
import cv2
import re
import numpy as np
import easyocr
reader = easyocr.Reader(['en'])
class Passportocr():

  def __init__(self):
    self.reader = reader

  def backText(self,inputtext):
    print("Processing of back text started")
    fname = None
    mname = None
    sname = None
    add = None

    try:
      temp = inputtext[inputtext.find(r'Father'):]
      temp = temp.split(",")
      fname = temp[1]
    except:
      fname = None

    try:
      temp = inputtext[inputtext.find(r'Mother'):]
      temp = temp.split(",")
      mname = temp[1]
    except:
      mname = None

    try:
      temp = inputtext[inputtext.find(r'Spouse'):]
      temp = temp.split(",")
      if "Address" in temp[1]:
        sname = None
      else:
        sname = temp[1]
    except:
      mname = None
      
    try:
      temp = inputtext[inputtext.find(r'Address'):inputtext.find(r'INDIA')]
      temp = temp.split(",")
      add = " ".join(temp)
    except:
      add = None

    dic = {
      "Fathername" : fname,
      "MotherName" : mname,
      "Spousename" : sname,
      "address": add
      }
    return dic


  def frontText(self,inputtext):
    print("processing front text started")
    temp =self.fixLetters(inputtext)
    pob = temp[temp.find(r'Place'):]
    pob = pob.split(",")
    if len(pob) >= 2:
      self.POB = pob[1].upper()
    else:
      self.POB = " "
      print("Not able to extract Place of Birth")
    try:

      lines = [x for x in inputtext.split(",") if re.search(r'<<',x)]

      if len(lines[0]) < 35:
        return ({},0)
      if len(lines[1]) < 28:
        return ({},0)
      self.nat = lines[0][2:5].upper()
      # print(iss_country)
      temp = re.findall(r'\w+', lines[0][5:43])
      self.sur_name = self.fixLetters(temp[0]).upper()
      # print(sur_name)
      self.first_name = " ".join([self.fixLetters(x) for x in temp[1:]]).upper()
      # print(first_name)
      self.pp_no = self.removeJunk(lines[1][:9])
      # print(pp_no)
      self.dob = self.fixDigits(lines[1][13:19])
      # print(dob)
      self.sex = self.fixLetters(lines[1][20]).upper()
      # print(sex)
      self.exp = self.fixDigits(lines[1][21:27])
      dic = {
        "Surname" : self.sur_name,
        "First Name" : self.first_name,
        "Date Of Birth(YYMMDD)" : self.dob,
        "Place of Birth": self.POB,
        "Passport Number" : self.pp_no,
        "Expiry Date(YYMMDD)" : self.exp,
        "Nationality" : self.nat,		
        "Sex" : self.sex,        
      }

    except:
      print("Image is Not clear to extract detail")
      dic = {"Surname" : None,
        "First Name" : None,
        "Date Of Birth(YYMMDD)" : None,
        "Place of Birth": None,
        "Passport Number" : None,
        "Expiry Date(YYMMDD)" : None,
        "Nationality" : None,		
        "Sex" : None}

    return dic

  def removeJunk(self,txt):
    return txt.replace('<','').replace(',','')
  def fixDigits(self,txt):
    data = [('A','-'),('B',8),('C','-'),('D',0),('E',3),('F',7),('G',6),('H',8),('I',1),('J','-'),('K',8),('L','-'),('M','-'),('N','-'),('O',0),('P',9),('Q',2),('R','-'),('S',5),('T',7),('U',0),('V','-'),('W','-'),('X','-'),('Y',5),('Z',2)]
    for t in data:
      txt = txt.replace(t[0], str(t[1]))
    return txt


  def fixLetters(self,txt):
    data = [(0,'O'),(1,'I'),(2,'Z'),(3,'-'),(4,'A'),(5,'S'),(6,'G'),(7,'T'),(8,'B'),(9,'-')]
    for t in data:
      txt = txt.replace(str(t[0]), t[1])
    return txt

  def detect_text_blocks(self,img_path):
      detection_result = self.reader.detect(img_path,
                                  width_ths=0.9, 
                                  mag_ratio=1.5
                                  )
      text_coordinates = detection_result[0][0]
      return text_coordinates

  def get_detail(self,img_front,img_back=None):
    print("Exctraction started")

    try:
      text_coordinates = self.detect_text_blocks(img_front)
      recognition_results = self.reader.recognize(img_front,
                                    horizontal_list=text_coordinates,
                                    free_list=[]
                                    )
      front_text = [txt[1] for txt in recognition_results]
      front_text = ",".join(front_text)
      print("text extracted from front image")

    except:
      print("front Image is Not clear to extract detail")
      front_text = " "

    dic_front = self.frontText(front_text)

    if img_back == None:
      print("Back image not received")    
      dic_back = {"Fathername" : None,
              "MotherName" : None,
              "Spousename" : None,
              "address": None
              }

    else:
      print("processging of back image started")
      try:
        text_coordinates = self.detect_text_blocks(img_back)
        recognition_results = self.reader.recognize(img_back,
                                      horizontal_list=text_coordinates,
                                      free_list=[]
                                      )
        back_text = [txt[1] for txt in recognition_results]
        back_text = ",".join(back_text)
        dic_back = self.backText(back_text)

      except:
        print("back Image is Not clear to extract detail")
        dic_back = {"Fathername" : None,
              "MotherName" : None,
              "Spousename" : None,
              "address": None
              }
      print("front and back text processed")
      dic_front.update(dic_back)
      print(" front and back dict combined")
    return dic_front

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [84]:
img_front = "/content/passport1.jpg"
img_back = "/content/Passport_back.JPG"

passport = Passportocr()
text = passport.get_detail(img_front,img_back)
text


Exctraction started
text extracted from front image
processing front text started
Not able to extract Place of Birth
processging of back image started
Processing of back text started
front and back text processed
 front and back dict combined


{'Date Of Birth(YYMMDD)': '590923',
 'Expiry Date(YYMMDD)': '211010',
 'Fathername': 'MAHAN SInGH',
 'First Name': 'SITA MAHA LAKSHMI',
 'MotherName': 'JOGINDER KAUR',
 'Nationality': 'IND',
 'Passport Number': 'J8369854',
 'Place of Birth': ' ',
 'Sex': 'F',
 'Spousename': 'BALJIT KAUR GILL',
 'Surname': 'RAMADUGULA',
 'address': 'Address Vpo  DhamOT KALAN TEHSIL PAYAL KHANNA (LUDhIANA) PIN:141413 PUnJaB  '}